In [29]:
from __future__ import print_function, division, unicode_literals
import six
import os
from os.path import join
import json
from codecs import open
from collections import defaultdict
from operator import itemgetter
import nltk
import numpy as np
from nltk.corpus import stopwords
import re
import codecs

import spacy

from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("english")

In [30]:
#import spacy.en

In [31]:
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english') + '. , ! ? ; - : —'.split())

In [32]:
#nlp = spacy.en.English()

In [33]:
def tokenize(text):
    return [stemmer.stem(w) for w in nltk.word_tokenize(text.lower()) if w not in stopwords]
    #return [stemmer.stem(w.text) for w in nlp(text) if not w.is_stop and not w.is_punct and not w.is_space]

In [34]:
tokenize("hello i am! —")

[u'hello']

In [35]:
"die" in stopwords

False

In [36]:
tokenize("Caps - acid")

[u'cap', u'acid']

In [37]:
import lucene

In [38]:
from org.apache.lucene.analysis.standard import StandardAnalyzer
from org.apache.lucene.document import Document, Field
from org.apache.lucene.index import IndexWriter, IndexWriterConfig, IndexReader
from org.apache.lucene.search import IndexSearcher
from org.apache.lucene.search import Sort, SortField
from org.apache.lucene.queryparser.classic import QueryParser
from org.apache.lucene.store import SimpleFSDirectory
from org.apache.lucene.util import Version

In [39]:
from java.io import File

In [40]:
DATA_DIR = join(os.environ['HOME'], 'data/allen-ai-challenge')
WIKI_DIR = join(DATA_DIR, 'wiki_dump')
CK12_DIR = join(DATA_DIR, 'ck12_dump')
WIKI_CK12_DIR = join(DATA_DIR, 'wiki_subset')
TRAINING_SET = join(DATA_DIR, 'training_set.tsv')
VALIDATION_SET = join(DATA_DIR, 'validation_set.tsv')
TRAINING_SET_MERGED = join(DATA_DIR, 'training_set_merged.tsv')
# INDEX_DIR = join(DATA_DIR, 'index-wiki-ck12')
# INDEX_DIR = join(DATA_DIR, 'index-ck12-stem')
INDEX_DIR = join(DATA_DIR, 'index-all-wiki_ck12_search4')
SUBMISSION = join(DATA_DIR, 'submissions/lucene_wiki_ck12_16jan.tsv')

In [41]:
lucene.initVM()

In [42]:
analyzer = StandardAnalyzer(Version.LUCENE_4_10_1)

In [43]:
writerConfig = IndexWriterConfig(Version.LUCENE_4_10_1, StandardAnalyzer())
writer = IndexWriter(SimpleFSDirectory(File(INDEX_DIR)), writerConfig)

wiki from ck12
----

In [44]:
%%time
for i, fn_short in enumerate(os.listdir(WIKI_CK12_DIR)):
    fn = join(WIKI_CK12_DIR, fn_short)
    with open(fn, encoding='utf-8', errors='ignore') as f:
        wiki_article = json.load(f)
        title, summary, content = wiki_article
        content = title + " " + summary + " " + content
        content = " ".join(tokenize(content))
        doc = Document()
        doc.add(Field("text", content, Field.Store.YES, Field.Index.ANALYZED))
        writer.addDocument(doc)

CPU times: user 7min, sys: 5.87 s, total: 7min 6s
Wall time: 7min 17s


In [45]:
writer.numDocs()

9423

ck12 parsed
---

In [46]:
%%time
for i, fn_short in enumerate(os.listdir(CK12_DIR)):
    fn = join(CK12_DIR, fn_short)
    with open(fn, encoding='utf-8', errors='ignore') as f:
        ck12_article = json.load(f)
        content = ck12_article["science"] + " " + ck12_article["concept"] + " " + '. '.join([paragraph for subtitle, paragraph in ck12_article['contents'].items()])
        content = " ".join(tokenize(content))
        doc = Document()
        doc.add(Field("text", content, Field.Store.YES, Field.Index.ANALYZED))
        writer.addDocument(doc)

KeyboardInterrupt: 

In [16]:
writer.numDocs()

1630

In [25]:
# for i, fn_short in enumerate(os.listdir(WIKI_DIR)):
#     fn = join(WIKI_DIR, fn_short)
#     with open(fn, encoding='utf-8', errors='ignore') as f:
#         wiki_article = json.load(f)
#         a, b, content = wiki_article
#         print(content)
# #         content = " ".join(tokenize(content[0]))
#         if i > 2:
#             break

from wiki science category
----

In [17]:
%%time
for i, fn_short in enumerate(os.listdir(WIKI_DIR)):
    fn = join(WIKI_DIR, fn_short)
    with open(fn, encoding='utf-8', errors='ignore') as f:
        wiki_article = json.load(f)
        _, summary, content = wiki_article
        content = summary + " " + content
        content = " ".join(tokenize(content))
        doc = Document()
        doc.add(Field("text", content, Field.Store.YES, Field.Index.ANALYZED))
        writer.addDocument(doc)

CPU times: user 6min 24s, sys: 5.16 s, total: 6min 29s
Wall time: 6min 27s


In [18]:
writer.numDocs()

13635

In [19]:
import telepot
def telegram_notify(msg):
    token = "178350795:AAFG7yae2SSt52GLek2bKS43oK7BaywWxRw"
    bot = telepot.Bot(token)
    b = bot.getMe()
    response = bot.getUpdates()
    bot.sendMessage(31747780, msg)

In [20]:
telegram_notify("lucene done!")

In [21]:
writer.numDocs()

13635

In [22]:
writer.close()

In [23]:
def iter_data(datafile, with_correct=True):
    with open(datafile, encoding='utf-8', errors='ignore') as f:
        next(f)
        for l in f:
            if with_correct:
                idd, q, correct, aa, ab, ac, ad = l.strip().split("\t")
            else:
                idd, q, aa, ab, ac, ad = l.strip().split("\t")
                correct = "no"
            q, aa, ab, ac, ad = [' '.join(tokenize(x)) for x in [q, aa, ab, ac, ad]]
            yield {"idd": idd, "q": q, "correct": correct, "aa": aa, "ab": ab, "ac": ac, "ad": ad}

In [24]:
from collections import defaultdict

In [25]:
%%time
res = defaultdict(list)
MAX = 100
docs_per_q = range(1, 20)

analyzer = StandardAnalyzer(Version.LUCENE_4_10_1)
reader = IndexReader.open(SimpleFSDirectory(File(INDEX_DIR)))
searcher = IndexSearcher(reader)

for row in iter_data(TRAINING_SET):
    queries = [row['aa'], row['ab'], row['ac'], row['ad']]
    queries = [row['q'] + ' ' + q  for q in queries]
    scores = defaultdict(list)
    for q in queries:
        query = QueryParser(Version.LUCENE_4_10_1, "text", analyzer).parse(re.sub("[^a-zA-Z0-9]"," ", q))
        #query = QueryParser(Version.LUCENE_30, "text", analyzer).parse(re.sub("[/^]", "\^", q))
        hits = searcher.search(query, MAX)
        doc_importance = [hit.score for hit in hits.scoreDocs]
        for n in docs_per_q:
            scores[n].append(sum(doc_importance[:n]))
      
    for n in docs_per_q:
        res[n].append(['A','B','C','D'][np.argmax(scores[n])] == row["correct"])

CPU times: user 23.6 s, sys: 2.03 s, total: 25.6 s
Wall time: 24.4 s


In [26]:
for x in sorted(res):
    print(x, np.mean(res[x]))

1 0.428
2 0.43
3 0.4328
4 0.4332
5 0.434
6 0.4384
7 0.4364
8 0.4336
9 0.4288
10 0.4304
11 0.4252
12 0.4248
13 0.4252
14 0.4248
15 0.424
16 0.4216
17 0.4224
18 0.4212
19 0.4184


In [36]:
%%time
MAX = 100
docs_per_q = 4

analyzer = StandardAnalyzer(Version.LUCENE_4_10_1)
reader = IndexReader.open(SimpleFSDirectory(File(INDEX_DIR)))
searcher = IndexSearcher(reader)

with open(SUBMISSION, "w") as s:
    s.write("id,correctAnswer\n")
    for row in iter_data(VALIDATION_SET, False):
        queries = [row['aa'], row['ab'], row['ac'], row['ad']]
        queries = [row['q'] + ' ' + q  for q in queries]
        scores = []
        for q in queries:
            query = QueryParser(Version.LUCENE_4_10_1, "text", analyzer).parse(re.sub("[^a-zA-Z0-9]"," ", q))
            #query = QueryParser(Version.LUCENE_30, "text", analyzer).parse(re.sub("[/^]", "\^", q))
            hits = searcher.search(query, MAX)
            doc_importance = [hit.score for hit in hits.scoreDocs]
            scores.append(sum(doc_importance[:docs_per_q]))
        guess = "ABCD"[np.argmax(scores)]
        s.write("%s,%s\n" % (row["idd"], guess))

CPU times: user 1min 14s, sys: 8.64 s, total: 1min 23s
Wall time: 1min 25s


Features
-----

In [71]:
FEATURES_LUCENE_TOP100_SCORES = join(DATA_DIR, 'features/lucene_top100.tsv')
FEATURES_LUCENE_ALL_SCORES = join(DATA_DIR, 'features/lucene_all.tsv')

In [72]:
%%time
MAX = 100
analyzer = StandardAnalyzer(Version.LUCENE_4_10_1)
reader = IndexReader.open(SimpleFSDirectory(File(INDEX_DIR)))
searcher = IndexSearcher(reader)

with open(FEATURES_LUCENE_TOP100_SCORES, "w") as fs:
    for row in iter_data(TRAINING_SET):
        queries = [row['aa'], row['ab'], row['ac'], row['ad']]
        queries = [row['q'] + ' ' + q  for q in queries]
        features = []
        for q in queries:
            query = QueryParser(Version.LUCENE_4_10_1, "text", analyzer).parse(re.sub("[^a-zA-Z0-9]"," ", q))
            hits = searcher.search(query, MAX)
            doc_importances = [str(hit.score) for hit in hits.scoreDocs]
            features.append(";".join(doc_importances))
        print(row["idd"], row["correct"], *features, file=fs, sep="\t")

CPU times: user 26.4 s, sys: 4.1 s, total: 30.5 s
Wall time: 32.1 s


In [98]:
sorter = Sort()
sorter.setSort(SortField("content", SortField.Type.STRING))

In [99]:
%%time
MAX = 10
analyzer = StandardAnalyzer(Version.LUCENE_4_10_1)
reader = IndexReader.open(SimpleFSDirectory(File(INDEX_DIR)))
searcher = IndexSearcher(reader)

with open(FEATURES_LUCENE_ALL_SCORES, "w") as fs:
    for row in iter_data(TRAINING_SET):
        queries = [row['aa'], row['ab'], row['ac'], row['ad']]
        queries = [row['q'] + ' ' + q  for q in queries]
        features = []
        for q in queries:
            query = QueryParser(Version.LUCENE_4_10_1, "text", analyzer).parse(re.sub("[^a-zA-Z0-9]"," ", q))      
            hits = searcher.search(query, MAX, sorter)
            doc_importances = [str(hit.score) for hit in hits.scoreDocs]
            features.append(";".join(doc_importances))
            print(doc_importances)
            break
        print(row["idd"], row["correct"], *features, file=fs, sep="\t")
        break

['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan']
CPU times: user 22.3 ms, sys: 3.8 ms, total: 26.1 ms
Wall time: 19.7 ms
